# Analyze Results

In [ ]:
%run ../acmodel/plot.py
%matplotlib ipympl
import pandas as pd
pd.set_option('display.max_colwidth', None)
from collections import Counter
import math
import numpy as np
import torch

# mega4 (balancing by phone frame freq)

In [ ]:
!ls -lrt mega[46]*.tsv

In [ ]:
tsvs = !ls mega[45]*.tsv
gr = []
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    sss = "".join([s for s in df.targets.values])
    c=Counter(sss)
    s = sorted(c.values(), reverse=True)
    logs = [math.log(x) for x in s]
    gr.append(torch.tensor(logs))
gr = torch.stack(gr)

plot_fun(gr.T)

In [ ]:
plot_fun(gr[:,25])

# mega3 training analysis (frequent phones getting everything)

In [ ]:
!ls -lrt mega3*.tsv

In [ ]:
df = pd.read_csv("mega3_training_0014.tsv", sep="\t", keep_default_na=False)

In [ ]:
c=Counter("".join([s for s in df.targets.values]))

In [ ]:
-torch.tensor([count for phone, count in sorted(i for i in c.items())]).log()

In [ ]:
import pandas as pd
from collections import Counter
def b_log_corrections(tsv_file):
    """
    Compute log(b()) additive correction needed to suppress very frequent
    phones and boost rare ones.
    """
    df = pd.read_csv(tsv_file, sep="\t", keep_default_na=False)
    c=Counter("".join([s for s in df.targets.values]))
    return -torch.tensor([count for phone, count in sorted(i for i in c.items())]).log()

In [ ]:
b_log_corrections("mega3_training_0014.tsv")

In [ ]:
tsvs = !ls mega3*.tsv
gr = []
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    sss = "".join([s for s in df.targets.values])
    c=Counter(sss)
    s = sorted(c.values(), reverse=True)
    logs = [math.log(x) for x in s]
    gr.append(torch.tensor(logs))
gr = torch.stack(gr)

plot_fun(gr.T)

# mega training analysis (just-sil recordings proliferating)

In [ ]:
df = pd.read_csv("mega_training_0004.tsv", sep="\t", keep_default_na=False)

In [ ]:
df

In [ ]:
def troubling_alignmet(s):
    """
    Count cases of a 1-frame only for state alignment (these are suspicious).
    Overall sum of troubling alignments hints quality of the acoustic model.
    """
    troubles = ""
    for i in range(1,len(s)-1):
        if s[i-1]!=s[i]!=s[i+1]:
            troubles += s[i]
    return troubles

In [ ]:
df.targets.values

In [ ]:
tsvs = !ls mega*.tsv
tsvs

In [ ]:
tsvs = !ls mega*.tsv
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    print(len("".join([troubling_alignmet(s) for s in df.targets.values])) / len("".join([s for s in df.targets.values])))
    sss = "".join([s for s in df.targets.values])

In [ ]:
from collections import Counter
c=Counter(sss)
c

In [ ]:
tsvs = !ls mega*.tsv
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    sss = "".join([s for s in df.targets.values])
    c=Counter(sss)
    print([c[x] for x in ['G', 'E', 'Z', 'Ž','A', '?', 'e', 's', 'ř', 'Ř', '|']])

In [ ]:
tsvs = !ls mega*.tsv
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    sss = "".join([s for s in df.targets.values])
    c=Counter(sss)
    print([c[x] for x in ['p', 't', 'k', 'g']])

In [ ]:
vowels_and_diphtongs = "AEOaeiouyáéíóúůýě"

In [ ]:
tsvs = !ls mega*.tsv
for t in tsvs:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    sss = "".join([s for s in df.targets.values])
    
    ccc = "".join(['|' if s=='|' else 'v' if s in vowels_and_diphtongs else 'c' for ss in df.targets.values for s in ss])
    c=Counter(ccc)
    print(c, end=''); print("  %.2f" % (c['v']/c['c']) )

In [ ]:
tsvs = !ls mega*.tsv
cc = []
for t in tsvs[-5], tsvs[-1]:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    #print(len("".join([troubling_alignmet(s) for s in df.targets.values])) / len("".join([s for s in df.targets.values])))
    sss = "".join([s for s in df.targets.values])
    cc.append(Counter(sss))

In [ ]:
for p in cc[0]:
    print(f"{p}: {cc[0][p]} -> {cc[1][p]}  {'%.1f' % (cc[1][p] / cc[0][p] * 100 - 100)} %")

## Find wavs with most added silence

In [ ]:
tsvs = !ls mega*.tsv
cc = []
for t in tsvs[5], tsvs[-1]:
    df = pd.read_csv(t, sep="\t", keep_default_na=False)
    df['sil'] = [len("".join(['|' if c=='|' else '' for c in s])) for s in df.targets.values]
    df['tot'] = [len(s) for s in df.targets.values]
    cc.append(df)

In [ ]:
cc[1]